# Extract accessions from Excel table:
Read the table:

In [1]:
import natsort

import pandas as pd

table = pd.read_excel('Supplementary_Table_1.xlsx',
                      engine='openpyxl')

Get BioProject of interest:

In [2]:
prj = 'PRJNA612766'

acc_df = (
    table.query('BioProject == @prj')
    [['Run', 'Sample Name', 'Collection_Date', 'Host', 'collected_by']]
    )

acc_df.groupby(['Host', 'collected_by', 'Collection_Date']).size()

Host          collected_by                         Collection_Date
Homo sapiens  Renmin Hospital of Wuhan University  12-Feb-2020         31
                                                   15-Jan-2020          2
                                                   30-Jan-2020         96
Plasmid       Aisi Fu                              12-Feb-2020        112
dtype: int64

Get accessions of interest for human samples:

In [3]:
acc_of_interest_df = (
 acc_df
 .query('Host == "Homo sapiens"')
 .query('not `Sample Name`.str.startswith("PC")')
 .query('not `Sample Name`.str.startswith("NC")')
 .assign(sample=lambda x: x['Sample Name'].str.split('-').str[0])
 .query('sample != "respiratory viruses"')
 # exclude negatives
 .query('sample not in ["D2", "F12", "D10", "A5", "H3", "A10"]')
 # exclude inconclusives
 .query('sample not in ["A3", "A7", "A8", "G6", "B5"]')
 .groupby(['sample', 'collected_by', 'Collection_Date'], as_index=False)
 .aggregate(accessions=pd.NamedAgg('Run', lambda g: list(g)))
 .sort_values('sample', key=natsort.natsort_keygen())
 .assign(collection_date=lambda x: x['Collection_Date'].where(x['sample'].str.startswith('R'), 'early in epidemic'))
 .drop(columns='Collection_Date')
 .reset_index(drop=True)
 )

acc_of_interest_df

,sample,collected_by,accessions,collection_date
0,A1,Renmin Hospital of Wuhan University,"[SRR11313282, SRR11313334]",early in epidemic
1,A2,Renmin Hospital of Wuhan University,"[SRR11313271, SRR11313461]",early in epidemic
2,A4,Renmin Hospital of Wuhan University,"[SRR11313417, SRR11313428]",early in epidemic
3,A6,Renmin Hospital of Wuhan University,"[SRR11313479, SRR11313490]",early in epidemic
4,A9,Renmin Hospital of Wuhan University,"[SRR11313345, SRR11313356]",early in epidemic
5,A11,Renmin Hospital of Wuhan University,"[SRR11313455, SRR11313456]",early in epidemic
6,A12,Renmin Hospital of Wuhan University,"[SRR11313473, SRR11313474]",early in epidemic
7,B1,Renmin Hospital of Wuhan University,"[SRR11313453, SRR11313454]",early in epidemic
8,B2,Renmin Hospital of Wuhan University,"[SRR11313451, SRR11313452]",early in epidemic
9,B3,Renmin Hospital of Wuhan University,"[SRR11313448, SRR11313449]",early in epidemic


Write to YAML for configuration file:

In [4]:
lines = ['# -----------------------------------------------------------------',
         f"# BioProject {prj}",
         '# -----------------------------------------------------------------',
         ]
for _, row in acc_of_interest_df.iterrows():
    d = row.to_dict()
    lines += [f"{d['sample']}_{prj}:",
              f"  accessions: {d['accessions']}",
              f"  collected_by: {d['collected_by']}",
              f"  collection_date: {d['collection_date']}"
              ]
with open(f"for_config.yml", 'w') as f:
    f.write('  ' + '\n  '.join(lines))

Get accessions of interest for plasmid samples, just keeping the highy copy ones for now:

In [5]:
acc_df_plasmid = (
    acc_df
    .query('Host == "Plasmid"')
    .query('`Sample Name`.str.startswith("1000cp") or `Sample Name`.str.startswith("3000cp")')
    .assign(sample=lambda x: 'plasmid-' + x['Sample Name'])
    .groupby(['sample', 'collected_by'], as_index=False)
    .aggregate(accessions=pd.NamedAgg('Run', lambda g: list(g)))
    .sort_values('sample', key=natsort.natsort_keygen())
    )

acc_df_plasmid

,sample,collected_by,accessions
1,plasmid-1000cp-replicate01-1h,Aisi Fu,[SRR11313331]
2,plasmid-1000cp-replicate01-2h,Aisi Fu,[SRR11313330]
4,plasmid-1000cp-replicate01-4h,Aisi Fu,[SRR11313329]
0,plasmid-1000cp-replicate01-10min,Aisi Fu,[SRR11313335]
3,plasmid-1000cp-replicate01-30min,Aisi Fu,[SRR11313332]
6,plasmid-1000cp-replicate02-1h,Aisi Fu,[SRR11313326]
7,plasmid-1000cp-replicate02-2h,Aisi Fu,[SRR11313325]
9,plasmid-1000cp-replicate02-4h,Aisi Fu,[SRR11313324]
5,plasmid-1000cp-replicate02-10min,Aisi Fu,[SRR11313328]
8,plasmid-1000cp-replicate02-30min,Aisi Fu,[SRR11313327]


Write to file:

In [6]:
lines = ['# -----------------------------------------------------------------',
         f"# BioProject {prj}",
         '# -----------------------------------------------------------------',
         ]
for _, row in acc_df_plasmid.iterrows():
    d = row.to_dict()
    lines += [f"{d['sample']}_{prj}:",
              f"  accessions: {d['accessions']}",
              f"  collected_by: {d['collected_by']}",
              ]
with open(f"for_config_plasmids.yml", 'w') as f:
    f.write('  ' + '\n  '.join(lines))